In [1]:
from usgs import api
import requests 
import json
import utils
import pandas as pd

In [2]:
URL = "https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/"
username = ''
password = ''
data = {'jsonRequest':'{"username": "'  + username +  '", "password": "'  + password +  '", "authType": "EROS", "catalogId": "EE"}'}
resp  = requests.post(URL+ "login", data=data)
api_key = resp.json()["data"]
print(api_key)

aa78839519164a7eb2db2e764a89ea0f


In [3]:
def find_images_ee(api_key,start,end,lat,lng) :
    print(start,end,lat,lng)
    datasets = [
        'NAIP',
        'HIGH_RES_ORTHO',
        'ORBVIEW3',
        'AVHRR_COMPOSITE',
        'AVHRR_ORBSEG',
        'AVHRR_1K_GLOBAL_COMP',
        'NON_COMMERCIAL_ORDERABLE_IK',
        'LANDSAT_8_C1',
        'LANDSAT_ETM_C1',
        'SPOT_CIB',
    ]
    r= []
    for dataset in datasets : 
        results = api.search(  
                                dataset, 'EE',
                                start_date=start, end_date=end,
                                lat =lat, lng = lng , distance =100 ,
                                api_key=api_key
        )

        for scene in results['data']['results']:
            print(scene,'\n\n')
            r.append(scene)
    
    
    return r


In [4]:
fires = pd.read_csv("./data/Fires_data/incendios2015.csv", sep=';' , encoding="utf-8",na_values="nan")
fires.head()
incendio = fires['incendio']==1
falso_alarme = fires['falso_alarme']==0 
falso_alarme_tipo = fires['tipo']!='Falso Alarme'
fogacho = fires['fogacho']==0
fires = fires[ incendio & falso_alarme & falso_alarme_tipo & fogacho]
fires.drop([
               'ano', 
               'codigo_sgif',
               'codigo_anpc',
               'data_primeira_intervencao',
               'hora_primeira_intervencao',
               'fonte_alerta',
               'nut',
               'ine',
               'causa', 'reacendimento' , 'agricola',
               'queimada','falso_alarme','fogacho','incendio','perimetro','aps', 'regiao_prof', 'ugf' 
          ],axis=1, inplace=True)
fires = fires.sort_values('area_total',  ascending=False)
fires.dropna()
fires.head()

C:\ProgramData\Anaconda3\envs\geo_py37\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tipo,distrito,concelho,freguesia,local,x,y,lat,lon,data_alerta,hora_alerta,data_extincao,hora_extincao,area_povoamento,area_mato,area_agricola,area_pov_mato,area_total,tipo_causa
21390,Florestal,Viseu,Mangualde,PÃ³voa de CervÃ£es,ContenÃ§as de Baixo,237389.0,398832.0,40:33:26.5068000000082'',7:41:29.922000000001'',06/08/2015,15:59:00,07/08/2015,07:33:00,29.89,955.11,0,985,985,NaN
6551,Florestal,Guarda,MÃªda,Ranhados,Pinhal das Forcas,267527.0,448034.0,40:59:54.643199999993'',7:19:49.9692000000005'',30/06/2015,14:34:00,01/07/2015,00:00:00,55.50,35,7.5,90.5,98,Intencional
7081,Florestal,Viseu,Armamar,SÃ£o Cosmado,Regadas,241299.0,454999.0,41:3:47.069999999992'',7:38:30.4511999999989'',30/03/2015,14:30:00,31/03/2015,02:54:00,24.85,72.89,0,97.74000000000001,97.74,Negligente
3947,Florestal,Castelo Branco,CovilhÃ£,Verdelhos,Sernadas,252322.0,375942.0,40:21:2.06999999999709'',7:31:2.07479999999883'',31/03/2015,09:18:00,01/04/2015,04:04:00,0.00,97.404,0,97.404,97.404,Negligente
21637,Florestal,Braga,GuimarÃ£es,SÃ£o Torcato,BOAVISTA,189451.0,501843.0,41:29:9.29040000000384'',8:15:34.1676000000021'',09/08/2015,11:20:00,10/08/2015,03:33:00,70.00,27,0,97,97,Desconhecida


In [5]:
head = fires.head(20)
results2015={ }
for index, row in head.iterrows():
    lat =utils.dms_to_decimal(row["lat"],"W")
    lon =utils.dms_to_decimal(row["lon"],"N")
    
    sensing_date_FROM = utils.dmy_to_ymd( row["data_alerta"] )
    sensing_date_TO = utils.dmy_to_ymd(  row["data_extincao"] )
    print('----------------------------------------->>>>>>>>>')
    
    r = find_images_ee(api_key,sensing_date_FROM,sensing_date_TO,lat,lon)
    results2015[index] = r
    print('\n\n')

----------------------------------------->>>>>>>>>
2015-08-06 2015-08-07 40.557363 -7.691645
{'acquisitionDate': '2015-08-06', 'startTime': '2015-08-06', 'endTime': '2015-08-06', 'spatialFootprint': {'type': 'Polygon', 'coordinates': [[[-8.21855, 39.67057], [-6.06112, 39.26294], [-5.48636, 40.97737], [-7.69886, 41.3872], [-8.21855, 39.67057]]]}, 'sceneBounds': '-8.21855,39.26294,-5.48636,41.3872', 'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_8_c1/2015/203/032/LC08_L1TP_203032_20150806_20170406_01_T1.jpg', 'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&ordered=LC82030322015218LGN01&node=INVSVC', 'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_8_C1/LC82030322015218LGN01/INVSVC/', 'entityId': 'LC82030322015218LGN01', 'displayId': 'LC08_L1TP_203032_20150806_20170406_01_T1', 'cloudCover': None, 'metadataUrl': 'https://earthexplorer.usgs.gov/metadata/xml/12864/LC82030322015218LGN01/', 'fgdcMetadataUrl': 'https




----------------------------------------->>>>>>>>>
2015-07-22 2015-07-23 40.354502 -7.39825



----------------------------------------->>>>>>>>>
2015-03-14 2015-03-14 41.474535 -7.91709
{'acquisitionDate': '2015-03-14', 'startTime': '2015-03-14', 'endTime': '2015-03-14', 'spatialFootprint': {'type': 'Polygon', 'coordinates': [[[-9.33262, 41.11287], [-7.09883, 40.7874], [-6.54383, 42.37963], [-8.83258, 42.7133], [-9.33262, 41.11287]]]}, 'sceneBounds': '-9.33262,40.7874,-6.54383,42.7133', 'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_etm_c1/2015/204/031/LE07_L1TP_204031_20150314_20161028_01_T1.jpg', 'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_ETM_C1&ordered=LE72040312015073ASN00&node=INVSVC', 'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_ETM_C1/LE72040312015073ASN00/INVSVC/', 'entityId': 'LE72040312015073ASN00', 'displayId': 'LE07_L1TP_204031_20150314_20161028_01_T1', 'cloudCover': None, 'metadataUrl': 

In [6]:
biggest_fires = [  
                 ("2017-06-16T00:00:00.000Z", "2017-06-24T00:00:00.000Z" ,"39.95", "-8.233333" , "Pedrogão grande" ),
                 ("2018-08-03T00:00:00.000Z", "2018-08-17T00:00:00.000Z" ,"37.396944", "-8.588889" , "Serra de Monchique"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.116667", "-8.25" , "Lousã"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.05", "-7.95" , "Pampilhosa da Serra"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"39.747767", "-8.949711" , "Pinhal de Leiria"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.416667", "-7.7" , "Seia"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.488733", "-7.864655" , "Nelas"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.6", "-7.766667" , "Mangualde"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.491022", "-7.593597" , "Gouveia"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.360278", "-7.861667" , "Oliveira do Hospital"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.360345", "-8.029032" , "Tábua"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.302222", "-8.166111" , "Penacova"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.393056", "-8.233333" , "Mortágua"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.516667", "-8.083333" , "Tondela"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.269030", "-7.988616" , "Coja, Arganil"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.85", "-8.4" , "Vale de Cambra"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"39.800833", "-8.100278" , "Sertã"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"42.044444", "-8.443333" , "Monção"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.4", "-8.133333" , "Santa Comba Dão"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.2175", "-8.852778" , "Quiaios"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.316667", "-8.8" , "Tocha"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.433333", "-8.733333" , "Mira"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.55", "-8.683333" , "Vagos"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.6", "-8.666667" , "Ílhavo"),
                 ("2017-10-13T00:00:00.000Z", "2017-10-18T00:00:00.000Z" ,"40.433333", "-8" , "Carregal do Sal")
]
fires = pd.DataFrame(biggest_fires, 
         columns = ['datetime_from' , 'datetime_to', 'lat', 'lon' , "description"]
        ) 
fires

,datetime_from,datetime_to,lat,lon,description
0,2017-06-16T00:00:00.000Z,2017-06-24T00:00:00.000Z,39.95,-8.233333,Pedrogão grande
1,2018-08-03T00:00:00.000Z,2018-08-17T00:00:00.000Z,37.396944,-8.588889,Serra de Monchique
2,2017-10-13T00:00:00.000Z,2017-10-18T00:00:00.000Z,40.116667,-8.25,Lousã
3,2017-10-13T00:00:00.000Z,2017-10-18T00:00:00.000Z,40.05,-7.95,Pampilhosa da Serra
4,2017-10-13T00:00:00.000Z,2017-10-18T00:00:00.000Z,39.747767,-8.949711,Pinhal de Leiria
5,2017-10-13T00:00:00.000Z,2017-10-18T00:00:00.000Z,40.416667,-7.7,Seia
6,2017-10-13T00:00:00.000Z,2017-10-18T00:00:00.000Z,40.488733,-7.864655,Nelas
7,2017-10-13T00:00:00.000Z,2017-10-18T00:00:00.000Z,40.6,-7.766667,Mangualde
8,2017-10-13T00:00:00.000Z,2017-10-18T00:00:00.000Z,40.491022,-7.593597,Gouveia
9,2017-10-13T00:00:00.000Z,2017-10-18T00:00:00.000Z,40.360278,-7.861667,Oliveira do Hospital


In [7]:
results2017={ }
for index, row in fires.iterrows():
    lat =float(row["lat"])
    lon =float(row["lon"])
    
    sensing_date_FROM = utils.iso_to_ymd( row["datetime_from"] )
    sensing_date_TO = utils.iso_to_ymd(  row["datetime_to"] )
    
    print('----------------------------------------->>>>>>>>>')
    r = find_images_ee(api_key,sensing_date_FROM,sensing_date_TO,lat,lon)
    results2017[index] = r
    print('\n\n')

----------------------------------------->>>>>>>>>
2017-06-16 2017-06-24 39.95 -8.233333
{'acquisitionDate': '2017-06-23', 'startTime': '2017-06-23', 'endTime': '2017-06-23', 'spatialFootprint': {'type': 'Polygon', 'coordinates': [[[-9.8096, 39.68408], [-7.62126, 39.36596], [-7.08628, 40.96086], [-9.326, 41.28659], [-9.8096, 39.68408]]]}, 'sceneBounds': '-9.8096,39.36596,-7.08628,41.28659', 'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_etm_c1/2017/204/032/LE07_L1TP_204032_20170623_20170719_01_T1.jpg', 'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_ETM_C1&ordered=LE72040322017174NSG00&node=INVSVC', 'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_ETM_C1/LE72040322017174NSG00/INVSVC/', 'entityId': 'LE72040322017174NSG00', 'displayId': 'LE07_L1TP_204032_20170623_20170719_01_T1', 'cloudCover': None, 'metadataUrl': 'https://earthexplorer.usgs.gov/metadata/xml/12267/LE72040322017174NSG00/', 'fgdcMetadataUrl': 'https:

{'acquisitionDate': '2017-10-13', 'startTime': '2017-10-13', 'endTime': '2017-10-13', 'spatialFootprint': {'type': 'Polygon', 'coordinates': [[[-9.83212, 39.68148], [-7.64355, 39.36377], [-7.11024, 40.95801], [-9.35017, 41.2833], [-9.83212, 39.68148]]]}, 'sceneBounds': '-9.83212,39.36377,-7.11024,41.2833', 'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_etm_c1/2017/204/032/LE07_L1TP_204032_20171013_20171109_01_T1.jpg', 'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_ETM_C1&ordered=LE72040322017286NSG00&node=INVSVC', 'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_ETM_C1/LE72040322017286NSG00/INVSVC/', 'entityId': 'LE72040322017286NSG00', 'displayId': 'LE07_L1TP_204032_20171013_20171109_01_T1', 'cloudCover': None, 'metadataUrl': 'https://earthexplorer.usgs.gov/metadata/xml/12267/LE72040322017286NSG00/', 'fgdcMetadataUrl': 'https://earthexplorer.usgs.gov/fgdc/12267/LE72040322017286NSG00/save_xml', 'modifiedDate': '

{'acquisitionDate': '2017-10-13', 'startTime': '2017-10-13', 'endTime': '2017-10-13', 'spatialFootprint': {'type': 'Polygon', 'coordinates': [[[-9.83212, 39.68148], [-7.64355, 39.36377], [-7.11024, 40.95801], [-9.35017, 41.2833], [-9.83212, 39.68148]]]}, 'sceneBounds': '-9.83212,39.36377,-7.11024,41.2833', 'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_etm_c1/2017/204/032/LE07_L1TP_204032_20171013_20171109_01_T1.jpg', 'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_ETM_C1&ordered=LE72040322017286NSG00&node=INVSVC', 'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_ETM_C1/LE72040322017286NSG00/INVSVC/', 'entityId': 'LE72040322017286NSG00', 'displayId': 'LE07_L1TP_204032_20171013_20171109_01_T1', 'cloudCover': None, 'metadataUrl': 'https://earthexplorer.usgs.gov/metadata/xml/12267/LE72040322017286NSG00/', 'fgdcMetadataUrl': 'https://earthexplorer.usgs.gov/fgdc/12267/LE72040322017286NSG00/save_xml', 'modifiedDate': '




----------------------------------------->>>>>>>>>
2017-10-13 2017-10-18 40.302222 -8.166111
{'acquisitionDate': '2017-10-13', 'startTime': '2017-10-13', 'endTime': '2017-10-13', 'spatialFootprint': {'type': 'Polygon', 'coordinates': [[[-9.83212, 39.68148], [-7.64355, 39.36377], [-7.11024, 40.95801], [-9.35017, 41.2833], [-9.83212, 39.68148]]]}, 'sceneBounds': '-9.83212,39.36377,-7.11024,41.2833', 'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_etm_c1/2017/204/032/LE07_L1TP_204032_20171013_20171109_01_T1.jpg', 'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_ETM_C1&ordered=LE72040322017286NSG00&node=INVSVC', 'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_ETM_C1/LE72040322017286NSG00/INVSVC/', 'entityId': 'LE72040322017286NSG00', 'displayId': 'LE07_L1TP_204032_20171013_20171109_01_T1', 'cloudCover': None, 'metadataUrl': 'https://earthexplorer.usgs.gov/metadata/xml/12267/LE72040322017286NSG00/', 'fgdcMetadataUrl




----------------------------------------->>>>>>>>>
2017-10-13 2017-10-18 39.800833 -8.100278
{'acquisitionDate': '2017-10-14', 'startTime': '2017-10-14', 'endTime': '2017-10-14', 'spatialFootprint': {'type': 'Polygon', 'coordinates': [[[-8.26142, 39.67067], [-6.10361, 39.26343], [-5.52943, 40.9779], [-7.74233, 41.38731], [-8.26142, 39.67067]]]}, 'sceneBounds': '-8.26142,39.26343,-5.52943,41.38731', 'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_8_c1/2017/203/032/LC08_L1TP_203032_20171014_20171024_01_T1.jpg', 'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&ordered=LC82030322017287LGN00&node=INVSVC', 'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_8_C1/LC82030322017287LGN00/INVSVC/', 'entityId': 'LC82030322017287LGN00', 'displayId': 'LC08_L1TP_203032_20171014_20171024_01_T1', 'cloudCover': None, 'metadataUrl': 'https://earthexplorer.usgs.gov/metadata/xml/12864/LC82030322017287LGN00/', 'fgdcMetadataUrl': 'h

{'acquisitionDate': '2017-10-13', 'startTime': '2017-10-13', 'endTime': '2017-10-13', 'spatialFootprint': {'type': 'Polygon', 'coordinates': [[[-9.83212, 39.68148], [-7.64355, 39.36377], [-7.11024, 40.95801], [-9.35017, 41.2833], [-9.83212, 39.68148]]]}, 'sceneBounds': '-9.83212,39.36377,-7.11024,41.2833', 'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_etm_c1/2017/204/032/LE07_L1TP_204032_20171013_20171109_01_T1.jpg', 'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_ETM_C1&ordered=LE72040322017286NSG00&node=INVSVC', 'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_ETM_C1/LE72040322017286NSG00/INVSVC/', 'entityId': 'LE72040322017286NSG00', 'displayId': 'LE07_L1TP_204032_20171013_20171109_01_T1', 'cloudCover': None, 'metadataUrl': 'https://earthexplorer.usgs.gov/metadata/xml/12267/LE72040322017286NSG00/', 'fgdcMetadataUrl': 'https://earthexplorer.usgs.gov/fgdc/12267/LE72040322017286NSG00/save_xml', 'modifiedDate': '